In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/sber_risk_spark/lesson_1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/sber_risk_spark/lesson_1'
/content/drive/MyDrive/sber_risk_spark/lesson_1


**Занятие первое**

Начнем с простого. Многие знают что такое map и reduce операции, но все же для закрпеления мы их тут реализуем. Ах да, не забудем и про shuffle. Делать все будем на упрощенной задаче с word count для ознакомления с самим подходом.

На самом деле мы рассмптрим все в упрощенном виде, но это даст нам понимание, как можно через hadoop streaming, например, писать самописные map и reduce операции

! mapred streaming \
  -input /wiki/sample.jsonl \
  -output /word-count \
  -mapper "/opt/conda/bin/python3.6 mapper.py" \
  -reducer "/opt/conda/bin/python3.6 reducer.py" \
  -file mapper.py \
  -file reducer.py

Выше mapper.py и reducer.py это программы, которые выполняют одноименные операции нам потоком информации из jsonl файла, записывая ответ в файл word-count

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re 
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Давайте загрузим файл с текстом и посмотрим на него

In [ ]:
with open('spark_text.txt', 'rb') as f:
    data = f.readlines()
data = [text.decode() for text in data if text.decode() != '\r\n']    

In [ ]:
len(data)

60

Как бы мы сделали..
Надо немного почистить слова, а также сделать все в парадигме MapReduce. Понятно, что можно все написать проще, но мы ведь хотим понять, как это работает=)

Загрузим стоп слова, очистим от них текст, приведем к нижнему регистру, всем раздадим ключи

In [ ]:
stop_words = stopwords.words("english")
stop_words = set(stop_words)

In [ ]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

пунктуацию тоже полезно бы удалить

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def mapper_text(text):
    clean_text = re.sub(rf"[{string.punctuation}]", "", text)
    words = nltk.word_tokenize(clean_text)
    words_with_value = [(word.lower(), 1) for word in words 
                        if word not in stop_words]
    words_with_value = sorted(words_with_value, key=lambda x:x[0])
    return words_with_value

def create_chunks(shuffled_data):
    result = {}
    for idx, data in shuffled_data:
        if idx in result:
            result[idx].append(data)
        else:
            result[idx] = [data]
    return list(result.items())

def shuffle_text(mapper_result, n_nodes=5):
    shuffled_data = []
    for key, value in mapper_result:
        shuffled_data.append((hash(key)%n_nodes, (key, value)))
    shuffled_data = sorted(shuffled_data, key=lambda x: x[0])
    chunks = create_chunks(shuffled_data)
    return chunks


# на самом деле для reduce в жизни пишут иначе..не зря мы сортируем внутри map
#данные по ключам. Это нужно для избавления от этапа проверки ключа и поиска
def reduce_text(values_to_reduce):
    result = {}
    for key, value in values_to_reduce:
        if key in result:
            result[key] += 1
        else:
            result[key] = 1
    return result

Проверим, что все работает

Сначала map

In [ ]:
data[0]

'Apache Spark is an open-source unified analytics engine for large-scale data processing. Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance. Originally developed at the University of California, Berkeley AMPLab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since.\r\n'

In [ ]:
map_stage = mapper_text(data[0])

In [ ]:
map_stage

[('amplab', 1),
 ('analytics', 1),
 ('apache', 1),
 ('apache', 1),
 ('berkeley', 1),
 ('california', 1),
 ('clusters', 1),
 ('codebase', 1),
 ('data', 1),
 ('data', 1),
 ('developed', 1),
 ('donated', 1),
 ('engine', 1),
 ('entire', 1),
 ('fault', 1),
 ('foundation', 1),
 ('implicit', 1),
 ('interface', 1),
 ('largescale', 1),
 ('later', 1),
 ('maintained', 1),
 ('opensource', 1),
 ('originally', 1),
 ('parallelism', 1),
 ('processing', 1),
 ('programming', 1),
 ('provides', 1),
 ('since', 1),
 ('software', 1),
 ('spark', 1),
 ('spark', 1),
 ('spark', 1),
 ('tolerance', 1),
 ('unified', 1),
 ('university', 1)]

shuffle

In [ ]:
shuffle_stage = shuffle_text(map_stage, 5)

In [ ]:
shuffle_stage

[(0,
  [('amplab', 1),
   ('clusters', 1),
   ('codebase', 1),
   ('fault', 1),
   ('foundation', 1)]),
 (1,
  [('data', 1),
   ('data', 1),
   ('entire', 1),
   ('interface', 1),
   ('later', 1),
   ('maintained', 1),
   ('opensource', 1),
   ('parallelism', 1),
   ('processing', 1)]),
 (2,
  [('apache', 1),
   ('apache', 1),
   ('berkeley', 1),
   ('california', 1),
   ('donated', 1),
   ('largescale', 1),
   ('tolerance', 1),
   ('unified', 1)]),
 (3,
  [('developed', 1),
   ('implicit', 1),
   ('originally', 1),
   ('programming', 1),
   ('software', 1),
   ('spark', 1),
   ('spark', 1),
   ('spark', 1)]),
 (4,
  [('analytics', 1),
   ('engine', 1),
   ('provides', 1),
   ('since', 1),
   ('university', 1)])]

reduce

In [ ]:
reduce_text(shuffle_stage[2][1])

{'apache': 2,
 'berkeley': 1,
 'california': 1,
 'donated': 1,
 'largescale': 1,
 'tolerance': 1,
 'unified': 1}

Итак, осталось все рассчитать параллельно и собрать результаты

In [ ]:
from joblib import Parallel, delayed

In [ ]:
n_nodes = 5

Обернем в 1 функциию для удобства map и shuffle

In [ ]:
def map_shuffle(text, n_nodes):
    map_result = mapper_text(text)
    shuffle_result = shuffle_text(map_result, n_nodes)
    return shuffle_result

In [ ]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(map_shuffle)(sent, n_nodes) for sent in data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed:    2.1s
[Parallel(n_jobs=5)]: Done  25 tasks      | elapsed:    2.2s
[Parallel(n_jobs=5)]: Done  60 out of  60 | elapsed:    2.3s finished


In [ ]:
len(res)==len(data)
dict(res[0])
res[0]

[(0,
  [('amplab', 1),
   ('analytics', 1),
   ('california', 1),
   ('foundation', 1),
   ('software', 1),
   ('tolerance', 1),
   ('university', 1)]),
 (1,
  [('apache', 1),
   ('apache', 1),
   ('berkeley', 1),
   ('interface', 1),
   ('maintained', 1),
   ('provides', 1)]),
 (2,
  [('clusters', 1),
   ('data', 1),
   ('data', 1),
   ('developed', 1),
   ('donated', 1),
   ('engine', 1),
   ('fault', 1),
   ('implicit', 1),
   ('parallelism', 1)]),
 (3,
  [('codebase', 1),
   ('largescale', 1),
   ('originally', 1),
   ('processing', 1),
   ('unified', 1)]),
 (4,
  [('entire', 1),
   ('later', 1),
   ('opensource', 1),
   ('programming', 1),
   ('since', 1),
   ('spark', 1),
   ('spark', 1),
   ('spark', 1)])]

Сделаем что-то вроде перессылки, собирая все в словари и заодно посмотрим на сколько равномерно распределлиись наши слова

In [ ]:
shuffle_stage = {i:[] for i in range(5)}
for values in res:
    values = dict(values)
    for key in values.keys():
        shuffle_stage[key].extend(values[key])

In [ ]:
shuffle_stage

In [ ]:
for key in shuffle_stage.keys():
    print(f'{key}: number of words = {len(shuffle_stage[key])}')

0: number of words = 416
1: number of words = 351
2: number of words = 421
3: number of words = 383
4: number of words = 350


И последний этап - нужно сделать reduce

In [ ]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(reduce_text)(shuffle_stage[key]) for key in shuffle_stage.keys())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


In [ ]:
res[0].keys()

dict_keys(['analytics', 'california', 'codebase', 'data', 'developed', 'api67', 'dataset', 'faulttolerant', 'in', 'primary', 'still', 'technology', 'underlies', 'function', 'paradigm', 'rdds', 'reduce', 'spark', 'working', 'compared', 'developing', 'learning', 'orders', 'spark10', 'either', 'for', 'launch', 'manager', 'single', 'storage', 'system', 'pseudodistributed', 'used', 'connect', 'java', 'project', 'python', 'achieved', 'driver', 'filter', 'mirrors', 'passing', 'scala', 'type', 'besides', 'broadcast', 'functional', 'nodes', 'reductions', 'two', 'argument', 'operation', 'performs', 'typical', 'words', '20', 'afforded', 'called', 'compiletime', 'dataframes', 'fully', 'language', 'supported', 'batch', 'capability', 'code', 'ingests', 'streaming', 'thus', 'written', 'equal', 'flink21', 'kinesis', 'minibatches', 'sockets22', 'separate', 'according', 'alternating', 'architecture', 'least', 'machinelearning', 'mllib', 'scale', 'vowpal', 'based', 'because', 'database26', 'like', 'utili

Собираем результат

In [ ]:
result = {}
for partition in res:
    for key in partition.keys():
        if key in result:
            result[key] += partition[key]
        else:
            result[key] = partition[key]

In [ ]:
sorted(result.items(), key=lambda x: x[1], reverse=True)

[('spark', 110),
 ('apache', 56),
 ('data', 44),
 ('streaming', 36),
 ('distributed', 23),
 ('processing', 19),
 ('sql', 17),
 ('learning', 15),
 ('also', 15),
 ('rdd', 15),
 ('api', 14),
 ('structured', 13),
 ('cluster', 12),
 ('machine', 12),
 ('hadoop', 11),
 ('rdds', 10),
 ('interface', 10),
 ('mapreduce', 10),
 ('provides', 10),
 ('mllib', 9),
 ('programming', 9),
 ('core', 9),
 ('the', 9),
 ('use', 9),
 ('graph', 9),
 ('framework', 9),
 ('used', 8),
 ('python', 8),
 ('code', 8),
 ('tasks', 8),
 ('application', 8),
 ('support', 8),
 ('dataset', 7),
 ('in', 7),
 ('operations', 7),
 ('algorithms', 7),
 ('run', 7),
 ('it', 7),
 ('graphx', 7),
 ('deep', 7),
 ('scala', 6),
 ('two', 6),
 ('batch', 6),
 ('big', 6),
 ('abstraction', 6),
 ('including', 6),
 ('simple', 6),
 ('pipelines', 6),
 ('graphs', 6),
 ('map', 6),
 ('implementation', 6),
 ('developers', 6),
 ('analytics', 5),
 ('system', 5),
 ('java', 5),
 ('project', 5),
 ('using', 5),
 ('even', 5),
 ('across', 5),
 ('applications', 

Да, было бы проще все сделать иным кодом и в один проход, но целью было разобрать, как все это примерно работает под капотом на больших данных.

**Домашнее задание**

Посчитать количество рейтингов больше 4 для каждого фильма и вывести фильмы в порядке убывания количества этих оценок

In [ ]:
with open('user_ratedmovies.dat', 'rb') as f:
    data = f.readlines()
headers = data[0].decode().split('\t')[:3]
data = [row.decode().split('\t')[:3] for row in data[1:]]

In [ ]:
headers, data[:2]

(['userID', 'movieID', 'rating'], [['75', '3', '1'], ['75', '32', '4.5']])

In [ ]:
headers

['userID', 'movieID', 'rating']

In [ ]:
data[0]

['75', '3', '1']

In [ ]:
len(data)

855598

Shuffle делать не нужно, он работает сам "под копотом". Пишем только map и reduce + параллелим вычисления. Лучше задавать batch_size при распараллеливании, либо даже заранее все разбить на батчи для обучения, будет быстрее

In [ ]:
import pandas as pd


# check for cases when a user has several ratings for the same film
tmp = pd.DataFrame(data, columns=headers)
tmp[tmp.duplicated(subset=['userID', 'movieID'])]

,userID,movieID,rating


In [ ]:
def map_rating(row):
    return (row[1], 1) if row[2] > '4' else (row[1], 0)


def shuffle_rating(mapper_result, n_nodes=5):
    shuffled_data = {hash(mapper_result[0]) % n_nodes: (mapper_result[0], mapper_result[1])}
    return shuffled_data


def map_shuffle_rating(row, n_nodes):
    map_result = map_rating(row)
    shuffle_result = shuffle_rating(map_result, n_nodes)
    return shuffle_result


def reduce_rating(values_to_reduce):
    result = {}
    for key, value in values_to_reduce:
        if key in result:
            result[key] += value
        else:
            result[key] = value
    return result

In [ ]:
print(map_rating(data[1]))
shuffle_rating(map_rating(data[1]))

('32', 1)


{2: ('32', 1)}

In [ ]:
with Parallel(n_jobs=n_nodes, verbose=1, batch_size=5) as parallel:
    res = parallel(delayed(map_shuffle_rating)(row, n_nodes) for row in data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done 210 tasks      | elapsed:    1.2s
[Parallel(n_jobs=5)]: Done 960 tasks      | elapsed:    1.5s
[Parallel(n_jobs=5)]: Done 2210 tasks      | elapsed:    2.0s
[Parallel(n_jobs=5)]: Done 3960 tasks      | elapsed:    2.7s
[Parallel(n_jobs=5)]: Done 6210 tasks      | elapsed:    3.6s
[Parallel(n_jobs=5)]: Done 8960 tasks      | elapsed:    4.6s
[Parallel(n_jobs=5)]: Done 12210 tasks      | elapsed:    5.9s
[Parallel(n_jobs=5)]: Done 15960 tasks      | elapsed:    7.3s
[Parallel(n_jobs=5)]: Done 20210 tasks      | elapsed:    8.9s
[Parallel(n_jobs=5)]: Done 24960 tasks      | elapsed:   11.4s
[Parallel(n_jobs=5)]: Done 30210 tasks      | elapsed:   16.2s
[Parallel(n_jobs=5)]: Done 35960 tasks      | elapsed:   19.9s
[Parallel(n_jobs=5)]: Done 42210 tasks      | elapsed:   22.3s
[Parallel(n_jobs=5)]: Done 48960 tasks      | elapsed:   24.8s
[Parallel(n_jobs=5)]: Done 56210 tasks      | elaps

In [ ]:
res[:3]

[{3: ('3', 0)}, {1: ('32', 1)}, {4: ('110', 0)}]

Сделаем что-то вроде перессылки, собирая все в словари и заодно посмотрим на сколько равномерно распределлиись наши слова

In [ ]:
shuffle_stage = {i:[] for i in range(n_nodes)}
for values in res:
    for key, val in values.items():
        shuffle_stage[key] += [val]

In [ ]:
shuffle_stage

{0: [('163', 0),
  ('832', 1),
  ('996', 1),
  ('1036', 0),
  ('2571', 1),
  ('3889', 0),
  ('7007', 1),
  ('39052', 0),
  ('162', 1),
  ('260', 0),
  ('327', 0),
  ('480', 0),
  ('497', 0),
  ('728', 0),
  ('745', 1),
  ('800', 0),
  ('1079', 1),
  ('1085', 1),
  ('1097', 0),
  ('1129', 0),
  ('1136', 1),
  ('1179', 0),
  ('1198', 0),
  ('1206', 1),
  ('1214', 1),
  ('1217', 1),
  ('1220', 1),
  ('1225', 1),
  ('1232', 1),
  ('1243', 1),
  ('1253', 1),
  ('1258', 1),
  ('1259', 1),
  ('1287', 0),
  ('1288', 1),
  ('1306', 0),
  ('1343', 0),
  ('1346', 0),
  ('1374', 0),
  ('1617', 0),
  ('1931', 1),
  ('1960', 0),
  ('1965', 1),
  ('1968', 0),
  ('2009', 0),
  ('2108', 0),
  ('2109', 0),
  ('2321', 0),
  ('2402', 0),
  ('2420', 0),
  ('2435', 1),
  ('2455', 1),
  ('2529', 1),
  ('2692', 1),
  ('2747', 0),
  ('2791', 1),
  ('2912', 1),
  ('2968', 1),
  ('2997', 1),
  ('3070', 0),
  ('3198', 1),
  ('3338', 0),
  ('3359', 1),
  ('3363', 0),
  ('3503', 1),
  ('3527', 0),
  ('3608', 1),
  

In [ ]:
with Parallel(n_jobs=n_nodes, verbose=10) as parallel:
    res = parallel(delayed(reduce_rating)(val) for val in shuffle_stage.values())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    4.1s remaining:    6.1s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    7.6s remaining:    5.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   11.5s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   11.5s finished


После reduce все можно собрать в одном цикле, считаем, что данные переслали после на 1 машину и агрегируем

In [ ]:
result = {}
for partition in res:
    for key in partition.keys():
        if key in result:
            result[key] += partition[key]
        else:
            result[key] = partition[key]

result
sorted(result.items(), key=lambda x: x[1], reverse=True)

[('2571', 900),
 ('318', 887),
 ('296', 878),
 ('2959', 828),
 ('4993', 756),
 ('7153', 719),
 ('5952', 697),
 ('858', 690),
 ('50', 688),
 ('2858', 680),
 ('4226', 636),
 ('260', 607),
 ('593', 606),
 ('1196', 601),
 ('4973', 594),
 ('1198', 588),
 ('356', 588),
 ('527', 572),
 ('7361', 567),
 ('1136', 554),
 ('47', 532),
 ('2762', 507),
 ('1193', 477),
 ('1221', 470),
 ('541', 462),
 ('2028', 452),
 ('608', 451),
 ('750', 446),
 ('6874', 446),
 ('32', 436),
 ('32587', 431),
 ('1089', 430),
 ('2329', 422),
 ('110', 413),
 ('8961', 412),
 ('4878', 410),
 ('3578', 409),
 ('1197', 408),
 ('1210', 408),
 ('1206', 407),
 ('912', 396),
 ('1270', 393),
 ('589', 391),
 ('1214', 389),
 ('1213', 387),
 ('6016', 383),
 ('7438', 380),
 ('1208', 378),
 ('4306', 377),
 ('6539', 377),
 ('6377', 374),
 ('1291', 372),
 ('293', 371),
 ('1258', 370),
 ('33794', 369),
 ('3996', 367),
 ('4995', 367),
 ('6711', 366),
 ('2997', 361),
 ('1704', 359),
 ('1265', 357),
 ('924', 352),
 ('4027', 343),
 ('1080', 3